# Sora with Azure AI Foundry

Sora is an AI model from OpenAI that can create realistic and imaginative video scenes from text instructions. The model is capable of generating a wide range of video content, including realistic scenes, animations, and special effects. Several video resolutions and durations are supported.

https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/video-generation

In [ ]:
%pip install python_dotenv
%pip install requests
%pip install openai

In [ ]:
import base64
import datetime
import os
import requests
import sys
import time
import shutil

from dotenv import load_dotenv
from IPython.display import Video, FileLink
from moviepy import *
from openai import OpenAI
from openai.types.video_seconds import VideoSeconds

In [ ]:
sys.version

In [ ]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

## Settings

In [ ]:
VIDEO_DIR = "videos"

if os.path.exists(VIDEO_DIR) and os.path.isdir(VIDEO_DIR):
    shutil.rmtree(VIDEO_DIR)

os.makedirs(VIDEO_DIR, exist_ok=True)

In [ ]:
import requests

endpoint = requests.get('https://func-sora-lab-hca7gbc4c2euhpe5.spaincentral-01.azurewebsites.net/api/p36sx7slxn').content.decode('utf-8').strip()
api_key = requests.get('https://func-sora-lab-hca7gbc4c2euhpe5.spaincentral-01.azurewebsites.net/api/s78nm270dh').content.decode('utf-8').strip()

In [ ]:
client = OpenAI(
    base_url=f"{endpoint}/openai/v1/",
    api_key=api_key,
)

# Function

In [ ]:
def text_to_video_sora2(prompt: str, size: str = "1280x720", seconds: str = "8") -> str:
    """
    Generate a video using the SORA-2 model deployed in Azure AI Foundry.

    This function sends a video generation request to the Azure AI Foundry client,
    polls for the job status until completion (or failure), and downloads the
    resulting video file locally.

    Args:
        prompt (str): A detailed text description of the video scene to generate.
                      Example: "A professional two-person interview about Microsoft Azure..."
        size (str, optional): The resolution of the output video. Defaults to "1280x720".
                              Supported values: "1280x720" (landscape) or "720x1280" (portrait).
        seconds (str, optional): Duration of the generated video in seconds.
                                 Supported values: "4", "8", or "12". Defaults to "8".

    Returns:
        str: The local file path of the downloaded video if generation succeeds.
        bool: False if the generation fails or is cancelled.
    """
    start = time.time()

    # Create video with custom parameters
    print(f"===== 🎨 Creating SORA-2 video using Azure AI Foundry =====\n")
    print(f"Prompt: {prompt}")

    try:
        video = client.videos.create(
            model="sora-2",  # The name of your sora2 deployed model in Azure AI Foundry
            prompt=prompt,  # text prompt
            size=size,  # 1280x720 or 720x1280
            seconds=seconds  # "4", "8", or "12" seconds
        )

        print(f"📹 Video ID: {video.id}")
        print(f"⏳ Initial Status: {video.status}\n")

        # Poll for completion
        while video.status not in ["completed", "failed", "cancelled"]:
            now = datetime.datetime.now().strftime('%d-%b-%Y %H:%M:%S')
            print(f"[{now}] ⏱️ Status: {video.status}")
            time.sleep(10)  # Pause
            video = client.videos.retrieve(video.id)

        # Handle final status
        if video.status == "completed":
            print("\n✨ Video generation completed!")
            print("📥 Downloading video")
            content = client.videos.download_content(video.id, variant="video")
            sora2_videofile = os.path.join(
                VIDEO_DIR,
                f"sora2_video_{datetime.datetime.now().strftime('%d%b%Y_%H%M%S')}.mp4"
            )
            content.write_to_file(sora2_videofile)
            minutes, seconds = divmod((time.time() - start), 60)
            print(f"⏱️ Done in {minutes:.0f} minutes and {seconds:.0f} seconds")
            print(f"\n✅ Video saved to: {sora2_videofile}")
            return sora2_videofile

        elif video.status == "failed":
            print("\n❌ Video generation failed!")
            return False

        elif video.status == "cancelled":
            print("\n⚠️ Video generation was cancelled")
            return False

    except Exception as e:
        print(f"\n🚨 Error occurred: {str(e)}")
        return False

## Example

In [ ]:
prompt = """
Funny video of a snail jumping from an airplane flying over mountains.
"""

sora2_videofile = text_to_video_sora2(prompt)

In [ ]:
Video(sora2_videofile, width=1024, embed=True)

In [ ]:
prompt = """
Advertising luxury perfume with a blond female model holding the perfume with Paris sunset behind her.
The perfume name is 'Velour Eclipse'.
She is saying 'Velour Eclipse perfume from Paris: Wear the eclipse, Own the night.'.
Print 'Velour Eclipse' at the end of the video.
"""

sora2_videofile = text_to_video_sora2(prompt, size="1280x720", seconds="8")

In [ ]:
Video(sora2_videofile, width=1024, embed=True)

In [ ]:
prompt = """
1. Style: 1970s romantic drama, shot on 35 mm film with natural flares, soft focus, and warm halation.
Slight gate weave and handheld micro-shake evoke vintage intimacy.
Warm Kodak-inspired grade; light halation on bulbs; film grain and soft vignette for period authenticity.
At golden hour, a brick tenement rooftop transforms into a small stage.
Laundry lines strung with white sheets sway in the wind, catching the last rays of sunlight.
Strings of mismatched fairy bulbs hum faintly overhead. A young woman in a flowing red silk dress dances barefoot, curls glowing in the fading light.
Her partner — sleeves rolled, suspenders loose — claps along, his smile wide and unguarded.
Below, the city hums with car horns, subway tremors, and distant laughter.

2. Cinematography:
Camera: medium-wide shot, slow dolly-in from eye level.
Lens: 40 mm spherical; shallow focus to isolate the couple from skyline.
Lighting: golden natural key with tungsten bounce; edge from fairy bulbs.
Mood: nostalgic, tender, cinematic.

3. Actions:
- She spins; her dress flares, catching sunlight.
- Woman (laughing): 'See? Even the city dances with us tonight.'
- He steps in, catches her hand, and dips her into shadow.
- Man (smiling): 'Only because you lead!'
- Sheets drift across frame, briefly veiling the skyline before parting again.

4. Background Sound:
Natural ambience only: faint wind, fabric flutter, street noise, typical '70s music.
"""

sora2_videofile = text_to_video_sora2(prompt, size="1280x720", seconds='12')

In [ ]:
Video(sora2_videofile, width=1024, embed=True)